In [43]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, RepeatVector, TimeDistributed

### Definindo Paramêtros

In [44]:
data = 'train-mod.csv'
kval = 10
itertot = 40
sigma = 1.2
itergd = 300

### Calcula valor através da função escolhida

In [45]:
def transforminput(param, center):
    
    def rbfGaussiana(i, j, sigma):
        return np.exp(-(np.sum((param[i] - center[j])**2.0)**0.5) / sigma**2.0)
  
    def rbfMultiquadratica(i, j):
        return np.sqrt(np.sum(np.sum(param[i] - center[j])**2.0 + center[j]**2.0))/np.sum(center[j], axis=0)

    def rbfMultiquadraticaInversa(i, j):
        return np.sum(center[j], axis=0)/np.sqrt(np.sum(np.sum(param[i] - center[j])**2.0 + center[j]**2.0))
    
    newinput = np.zeros((len(param), len(center))).astype('float32')
    for i in range(len(param)):
        for j in range(len(center)):
            newinput[i,j] = rbfGaussiana(i, j, sigma)
    return newinput

In [46]:
def generatemodel(numparam):
    model = Sequential()
    model.add(Dense(1, input_dim=numparam, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [47]:
dataread = np.genfromtxt(data, delimiter=',')[1:,1:]

### Tratando entrada

In [48]:
alldata = []
for i in range(len(dataread)):
    if np.isnan(dataread[i,-2]):
        continue
    alldata.append(dataread[i])

alldata = np.asarray(alldata)

#dividing data

trainparam = alldata[:600,1:]
trainlabel = alldata[:600,0]
testparam = alldata[600:,1:]
testlabel = alldata[600:,0]

###############
#normalization#
###############

std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))
for i in range(len(trainparam[0])):
    std[i] = np.std(trainparam[:,i])
    rata[i] = np.mean(trainparam[:,i])
    trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
    testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

### KMeans

In [49]:
kmean = np.zeros((kval, len(trainparamnorm[0])))
for i in range(kval):
    for j in range(len(kmean[0])):
        kmean[i,j] = random.uniform(min(trainparamnorm[:,j]),max(trainparamnorm[:,j]))
#looping of real algorithm
distmin = np.zeros((len(trainparamnorm)))
for i in range(itertot):
    print ('iteração para: ', i)
    for j in range(len(distmin)):
        #determine euclid distance
        distall = np.sum((trainparamnorm[j] - kmean)**2.0, axis=1)**0.5
        distmin[j] = np.argmin(distall)
#search new k mean
for j in range(kval):
    clust = []
    for k in range(len(distmin)):
        if distmin[k] == j:
            clust.append(trainparamnorm[k])
    if len(clust) > 0:
        kmean[j] = np.mean(np.asarray(clust), axis=0)

iteração para:  0
iteração para:  1
iteração para:  2
iteração para:  3
iteração para:  4
iteração para:  5
iteração para:  6
iteração para:  7
iteração para:  8
iteração para:  9
iteração para:  10
iteração para:  11
iteração para:  12
iteração para:  13
iteração para:  14
iteração para:  15
iteração para:  16
iteração para:  17
iteração para:  18
iteração para:  19
iteração para:  20
iteração para:  21
iteração para:  22
iteração para:  23
iteração para:  24
iteração para:  25
iteração para:  26
iteração para:  27
iteração para:  28
iteração para:  29
iteração para:  30
iteração para:  31
iteração para:  32
iteração para:  33
iteração para:  34
iteração para:  35
iteração para:  36
iteração para:  37
iteração para:  38
iteração para:  39


In [50]:
#tranform our input
newinput = transforminput(trainparamnorm, kmean)

print(trainlabel)

[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.

In [51]:
mod = generatemodel(kval)
mod.fit(newinput, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)

Epoch 1/300
600/600 [==============================] - 0s 194us/step - loss: 0.2612 - acc: 0.4067
Epoch 2/300
600/600 [==============================] - 0s 43us/step - loss: 0.2583 - acc: 0.4067
Epoch 3/300
600/600 [==============================] - 0s 44us/step - loss: 0.2559 - acc: 0.4217
Epoch 4/300
600/600 [==============================] - 0s 37us/step - loss: 0.2536 - acc: 0.4117
Epoch 5/300
600/600 [==============================] - 0s 38us/step - loss: 0.2518 - acc: 0.4133
Epoch 6/300
600/600 [==============================] - 0s 41us/step - loss: 0.2498 - acc: 0.4600
Epoch 7/300
600/600 [==============================] - 0s 40us/step - loss: 0.2483 - acc: 0.5533
Epoch 8/300
600/600 [==============================] - 0s 37us/step - loss: 0.2468 - acc: 0.5950
Epoch 9/300
600/600 [==============================] - 0s 40us/step - loss: 0.2455 - acc: 0.6033
Epoch 10/300
600/600 [==============================] - 0s 50us/step - loss: 0.2443 - acc: 0.6083
Epoch 11/300
600/600 [======

600/600 [==============================] - 0s 35us/step - loss: 0.2184 - acc: 0.6550
Epoch 85/300
600/600 [==============================] - 0s 43us/step - loss: 0.2182 - acc: 0.6583
Epoch 86/300
600/600 [==============================] - 0s 35us/step - loss: 0.2180 - acc: 0.6583
Epoch 87/300
600/600 [==============================] - 0s 38us/step - loss: 0.2178 - acc: 0.6583
Epoch 88/300
600/600 [==============================] - 0s 41us/step - loss: 0.2177 - acc: 0.6567
Epoch 89/300
600/600 [==============================] - 0s 36us/step - loss: 0.2175 - acc: 0.6567
Epoch 90/300
600/600 [==============================] - 0s 37us/step - loss: 0.2173 - acc: 0.6567
Epoch 91/300
600/600 [==============================] - 0s 38us/step - loss: 0.2171 - acc: 0.6567
Epoch 92/300
600/600 [==============================] - 0s 43us/step - loss: 0.2169 - acc: 0.6567
Epoch 93/300
600/600 [==============================] - 0s 39us/step - loss: 0.2167 - acc: 0.6583
Epoch 94/300
600/600 [===========

600/600 [==============================] - 0s 46us/step - loss: 0.2065 - acc: 0.6817
Epoch 168/300
600/600 [==============================] - 0s 55us/step - loss: 0.2064 - acc: 0.6817
Epoch 169/300
600/600 [==============================] - 0s 33us/step - loss: 0.2063 - acc: 0.6817
Epoch 170/300
600/600 [==============================] - 0s 34us/step - loss: 0.2062 - acc: 0.6817
Epoch 171/300
600/600 [==============================] - 0s 38us/step - loss: 0.2061 - acc: 0.6817
Epoch 172/300
600/600 [==============================] - 0s 60us/step - loss: 0.2060 - acc: 0.6817
Epoch 173/300
600/600 [==============================] - 0s 44us/step - loss: 0.2059 - acc: 0.6817
Epoch 174/300
600/600 [==============================] - 0s 40us/step - loss: 0.2058 - acc: 0.6817
Epoch 175/300
600/600 [==============================] - 0s 48us/step - loss: 0.2057 - acc: 0.6817
Epoch 176/300
600/600 [==============================] - 0s 44us/step - loss: 0.2056 - acc: 0.6817
Epoch 177/300
600/600 [=

600/600 [==============================] - 0s 37us/step - loss: 0.1999 - acc: 0.6917
Epoch 250/300
600/600 [==============================] - 0s 34us/step - loss: 0.1999 - acc: 0.6883
Epoch 251/300
600/600 [==============================] - 0s 40us/step - loss: 0.1998 - acc: 0.6883
Epoch 252/300
600/600 [==============================] - 0s 35us/step - loss: 0.1998 - acc: 0.6900
Epoch 253/300
600/600 [==============================] - 0s 38us/step - loss: 0.1997 - acc: 0.6883
Epoch 254/300
600/600 [==============================] - 0s 37us/step - loss: 0.1996 - acc: 0.6883
Epoch 255/300
600/600 [==============================] - 0s 45us/step - loss: 0.1996 - acc: 0.6883
Epoch 256/300
600/600 [==============================] - 0s 39us/step - loss: 0.1995 - acc: 0.6883
Epoch 257/300
600/600 [==============================] - 0s 46us/step - loss: 0.1994 - acc: 0.6883
Epoch 258/300
600/600 [==============================] - 0s 42us/step - loss: 0.1994 - acc: 0.6883
Epoch 259/300
600/600 [=

In [52]:
newinputtest = transforminput(testparamnorm, kmean)

In [53]:
lifeprob = mod.predict(newinputtest)

### Armazenando os valores previstos

In [54]:
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
    if lifeprob[i] > 0.5:
        binpred[i] = 1.

### Calculando a acurácia

In [55]:
score = 0
for i in range(len(testlabel)):
    if binpred[i] == testlabel[i]:
        score += 1
accbin = float(score) / float(len(testlabel))

In [56]:
errorabs = abs(score-len(lifeprob))
error = np.sum(errorabs/len(testlabel), axis=0)

In [57]:
brierscore = 0
for i in range(len(testlabel)):
    brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))

In [58]:
for i in range(len(testlabel)):
    print (lifeprob[i], "-", testlabel[i])

[0.29081053] - 0.0
[0.48522216] - 1.0
[0.46419787] - 1.0
[0.27031285] - 0.0
[0.3347677] - 0.0
[0.26604438] - 0.0
[0.72292936] - 1.0
[0.287655] - 0.0
[0.30837962] - 1.0
[0.8025757] - 1.0
[0.33499163] - 0.0
[0.5929351] - 1.0
[0.34810662] - 0.0
[0.26474088] - 0.0
[0.28562003] - 0.0
[0.32344723] - 0.0
[0.42134622] - 0.0
[0.42441952] - 1.0
[0.32143146] - 0.0
[0.43739977] - 1.0
[0.83038116] - 1.0
[0.40387005] - 1.0
[0.6587901] - 1.0
[0.46098107] - 0.0
[0.28112346] - 0.0
[0.28109664] - 0.0
[0.38329935] - 1.0
[0.40103245] - 0.0
[0.43044853] - 1.0
[0.55574054] - 0.0
[0.37015095] - 0.0
[0.28102106] - 0.0
[0.27158475] - 0.0
[0.49330327] - 1.0
[0.34798878] - 1.0
[0.2663834] - 0.0
[0.3588058] - 0.0
[0.25216725] - 0.0
[0.39682835] - 1.0
[0.74207675] - 1.0
[0.42590845] - 1.0
[0.27344614] - 1.0
[0.26522756] - 0.0
[0.40744865] - 0.0
[0.38344574] - 0.0
[0.27158475] - 0.0
[0.60048074] - 1.0
[0.27687317] - 0.0
[0.2827918] - 0.0
[0.25441885] - 0.0
[0.4495088] - 0.0
[0.26560286] - 0.0
[0.36486328] - 0.0
[0.

In [59]:
print ("Accuracy: ", accbin, "Brier score:", brierscore[0])
print ("Error: ", error)

Accuracy:  0.7192982456140351 Brier score: 0.17955261
Error:  0.2807017543859649
